In [ ]:
import os
import base64
from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

prompt = """
트레이딩 카드 게임 스타일의 카드 일러스트를 생성하세요.

=== 카드 레이아웃 (시각적 구조) ===

┌─────────────────────────────────────────┐
│  [배경 이미지 - Layer 2 (전체 영역)]     │
│  ┌─────────────────────────────────┐   │
│  │                                 │   │
│  │  ⭕개발자  ⭐  엔카코  어둠⭕ │
│  │  ─────────────────────────────  │
│  │                                 │
│  │    [메인 캐릭터 이미지 - Layer 1]  │
│  │                                 │
│  │  ─────────────────────────────  │
│  │  [스킬 1] 디버깅                 │
│  │  • '몰입' 모드로 태세를 바꿉니다.            │
│  │                                 │
│  │  [스킬 2] 컴파일                 │
│  │  • 모든 개발을 종료합니다.                 │
│  │                                 │
│  │  ─────────────────────────────  │
│  │  "세상 모든 0 과 1 은 나의 손에...             "│
│  │                                 │
│  │  ⚔️ 1000  ❤️ 1000                   │
│  │                                 │
│  │  #001              default / ncaco.inc│
│  └─────────────────────────────────┘   │
└─────────────────────────────────────────┘
(모든 텍스트는 투명 배경 오버레이로 배경 위에 표시)

=== 카드 데이터 (구조화된 정보) ===

{
  "layout": {
    "layer2": {
      "type": "배경 이미지",
      "description": "카드 전체를 덮는 배경 이미지",
      "reference": "없음"
    },
    "layer1": {
      "type": "메인 캐릭터 이미지",
      "description": "배경 위 중앙에 배치되는 메인 캐릭터",
      "reference": "업로드된 캐릭터 이미지 스타일 참고"
    }
  },
  "header": {
    "type": "개발자",
    "rarity": "⭐",
    "cardName": "엔카코",
    "attribute": "어둠"
  },
  "skills": [
    {
      "name": "디버깅",
      "description": "'몰입' 모드로 태세를 바꿉니다."
    },
    {
      "name": "컴파일",
      "description": "모든 개발을 종료합니다."
    }
  ],
  "stats": {
    "attack": "1000",
    "health": "1000"
  },
  "description": "세상 모든 0 과 1 은 나의 손에...",
  "meta": {
    "cardNumber": "#001",
    "series": "default / ncaco.inc"
  }
}

=== 스타일 가이드 ===
- 트레이딩 카드 게임 스타일 (포켓몬카드, 원피스카드 등 참고)
- 모든 텍스트는 투명도가 높은 배경 위에 오버레이로 표시
- 배경 이미지가 카드 전체를 덮고, 그 위에 캐릭터와 텍스트가 배치됨
- 상세하고 전문적인 일러스트 품질
- 카드 비율: 5:7 (세로형, 400x560px 기준)
- 카드번호는 좌하단에 표시 (#001 형식)
- 제작사/시리즈 정보는 우하단에 표시
- 속성 표시는 카드 상단 우측 배지 형태로 강조
- 속성 표시는 글자 대신 아이콘/심볼로만 표현
- 스킬1과 스킬2 사이, 스킬2와 플레이버 텍스트 사이에 구분선을 표시하여 각 영역을 명확히 구분
"""

try:
    # GPT-Image-1 사용 (OpenAI의 최신 이미지 생성 모델)
    # 참고: GPT-Image-1는 프롬프트를 기반으로 고품질 이미지를 생성합니다.
    # 참조 이미지 스타일을 반영하려면 프롬프트에 스타일 설명을 포함하세요.
    
    print("이미지 생성 중... (GPT-Image-1 사용)")
    print("참고: GPT-Image-1는 고품질 이미지 생성에 최적화되어 있습니다.")
    
    result = client.images.generate(
        model="gpt-image-1",
        prompt=prompt,
        size="1024x1024",  # "1024x1024", "1536x1024" (landscape), "1024x1536" (portrait)
        quality="high",  # "low", "medium", "high", "auto" (기본값: "auto")
        n=1,  # 생성할 이미지 개수 (1-10)
        output_format="png",  # "png", "jpeg", "webp"
        # output_compression=80,  # webp/jpeg 사용 시 압축 레벨 (0-100)
        response_format="b64_json"  # "url" or "b64_json"
    )
    
    # Base64 형식으로 응답 받기
    if hasattr(result.data[0], 'b64_json') and result.data[0].b64_json:
        image_base64 = result.data[0].b64_json
        image_bytes = base64.b64decode(image_base64)
        
        # 이미지 저장
        output_filename = "trading-card-generated.png"
        with open(output_filename, "wb") as f:
            f.write(image_bytes)
        print(f"✅ 이미지가 성공적으로 생성되어 '{output_filename}'에 저장되었습니다.")
    else:
        # URL 형식으로 응답 받은 경우
        image_url = result.data[0].url
        print(f"✅ 이미지가 생성되었습니다. URL: {image_url}")
        print("URL에서 이미지를 다운로드하려면 requests 라이브러리를 사용하세요.")
        
except Exception as e:
    error_msg = str(e)
    print(f"❌ 오류 발생: {error_msg}")
    
    if "billing_hard_limit_reached" in error_msg or "Billing hard limit" in error_msg:
        print("\n" + "="*60)
        print("⚠️  결제 한도에 도달했습니다!")
        print("="*60)
        print("\n해결 방법:")
        print("1. OpenAI 계정(https://platform.openai.com)에 로그인")
        print("2. 'Billing' 또는 'Usage' 섹션으로 이동")
        print("3. 결제 한도를 확인하고 필요시 증가시키기")
        print("4. 또는 새로운 API 키 생성")
        print("\n참고: 현재 사용 중인 API 키의 결제 한도가 초과되었습니다.")
        print("="*60)
    elif "rate_limit" in error_msg.lower():
        print("\n⚠️  API 호출 제한에 도달했습니다. 잠시 후 다시 시도해주세요.")
    else:
        print(f"\n상세 오류 정보:")
        print(f"오류 타입: {type(e).__name__}")
        import traceback
        traceback.print_exc()

이미지 생성 중... (DALL-E 3 사용)
참고: DALL-E 3는 참조 이미지를 직접 받지 않지만, 프롬프트에 스타일 설명을 포함할 수 있습니다.
❌ 오류 발생: Error code: 400 - {'error': {'message': 'Billing hard limit has been reached', 'type': 'image_generation_user_error', 'param': None, 'code': 'billing_hard_limit_reached'}}

⚠️  결제 한도에 도달했습니다!

해결 방법:
1. OpenAI 계정(https://platform.openai.com)에 로그인
2. 'Billing' 또는 'Usage' 섹션으로 이동
3. 결제 한도를 확인하고 필요시 증가시키기
4. 또는 새로운 API 키 생성

참고: 현재 사용 중인 API 키의 결제 한도가 초과되었습니다.
